# Transformer的pythoch实现

## 数据预处理
使用transformer执行文本翻译任务，数据集选用英文和法语数据集

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from batch import *
from process import *
import numpy as np
import time

# 数据
src_file = 'data/english.txt'
trg_file = 'data/french.txt'
src_lang = 'en_core_web_sm'
trg_lang = 'fr_core_news_sm'
max_strlen = 80
batchsize = 1500
src_data, trg_data = read_data(src_file, trg_file)  # 一个包含所有源语言（英语）句子的字符串列表。154883
EN_TEXT, FR_TEXT = create_fields(src_lang, trg_lang)
train_iter, src_pad, trg_pad = create_dataset(src_data, trg_data, EN_TEXT, FR_TEXT, max_strlen, batchsize) # , 1 , 1
src_vocab = len(EN_TEXT.vocab) #13724 源语句的token数量
trg_vocab = len(FR_TEXT.vocab) #23469 目标语句的token数量


loading spacy tokenizers...
creating dataset and iterator... 
1089


In [2]:
# 获取训练集中的前2个batch，便于测试
sample_batches = []
for i, batch in enumerate(train_iter):
    sample_batches.append(batch)
    if i >= 0:  # 只取前1个batch
        break

# 打印第一个batch的源语言和目标语言内容（索引形式）
print("第一个batch的源语言（索引）:")
print(sample_batches[0].src)
print("第一个batch的目标语言（索引）:")
print(sample_batches[0].trg)
# shape为torch.Size([7, 214])，214句话，每句话有7个token
# 如果想要将索引还原为单词，可以这样做：
src_vocab_obj = EN_TEXT.vocab
trg_vocab_obj = FR_TEXT.vocab

def indices_to_words(indices, vocab):
    return [vocab.itos[idx] for idx in indices]

# 以第一个batch的第一句话为例，转换为单词
src_indices = sample_batches[0].src[:, 0]  # 第一句
trg_indices = sample_batches[0].trg[:, 0]  # 第一句

print("第一个batch第一句源语言（单词）:")
print(indices_to_words(src_indices, src_vocab_obj))
print("第一个batch第一句目标语言（单词）:")
print(indices_to_words(trg_indices, trg_vocab_obj))

第一个batch的源语言（索引）:
tensor([[  31,    3,    3,  ...,   13,   15,    3],
        [ 290,   49,   10,  ..., 2190,   11,   10],
        [  47,   12,   25,  ...,   14,  106,    9],
        ...,
        [  43,   95,   32,  ...,   23,  105,  106],
        [ 113, 1601,  125,  ...,  309,   84,  602],
        [   2,    2,    2,  ...,    2,    2,    2]])
第一个batch的目标语言（索引）:
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  32,    5,    5,  ...,   11,   24,    5],
        [  21,   84,  159,  ...,   18,   10,   14],
        ...,
        [ 129, 6919,   78,  ...,  321,  207,  674],
        [   4,    4,    4,  ...,    4,    4,    4],
        [   3,    3,    3,  ...,    3,    3,    3]])
第一个batch第一句源语言（单词）:
['she', 'married', 'him', 'for', 'his', 'money', '.']
第一个batch第一句目标语言（单词）:
['<sos>', 'elle', "l'", 'a', 'épousé', 'pour', 'son', 'argent', '.', '<eos>']


## 模型参数

In [3]:
d_model = 512
heads = 8
N = 6
dropout = 0.1

## Embedding

In [4]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, d_model) -> None:
        super().__init__()
        self.d_model = d_model
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

## Positional Encoding
![positional encoding](./picture/image1.png)

In [5]:
import math
from torch.autograd import Variable
class positionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_len=80, dropout=0.1) -> None:
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        #根据输入语句的token数和嵌入向量的维度构造位置编码矩阵
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** (i / d_model)))
                if i + 1 < d_model:
                    pe[pos, i + 1] = math.cos(pos / (10000 ** (i / d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe) 

    def forward(self, x):
        # 使得单词嵌入表示相对大一些
        x = x * math.sqrt(self.d_model)
        # 增加位置常量到单词嵌入表示中
        seq_len = x.size(1)
        x = x + Variable(self.pe[:, :seq_len, :], requires_grad=False)
        return self.dropout(x)

## Self-Attention Layer
对于输入的句子X，通过 WordEmbedding 得到该句子中每个字的字向量，同时通过Positional Encoding 得到所有字的位置向量，将其相加(维度相同，可以直接相加)，得到该字真正的向量表示。第t个字的向量记作$x_t$

接着我们定义三个矩阵 $W_Q$,$W_K$,$W_V$,使用这三个矩阵分别对所有的字向量进行三次线性变换，于是所有的字向量又衍生出三个新的向量$q_t$,$k_t$,$v_t$。我们将所有的q向量拼成一个大矩阵，记作查询矩阵Q，将所有的k向量拼成一个大矩阵，记作键矩阵K，将所有的v向量拼成一个大矩阵，记作值矩阵V(见下图)

![](./picture/image2.png)

接下来将Q和$K^T$相乘，得到注意力分数矩阵，其中的每一行代表一个query(当前token)对所有的key的相关性分数，然后除以$\sqrt{d_k}$(这是论文中提到的一个 trick),经过 softmax后,每一行是一个概率分布，表示该query“关注”每个key的程度，再乘以 V 得到输出，此时的每一行代表当前query（比如当前单词）在全局信息加权融合后的新表示，这个新表示综合了序列中所有位置的信息。

![](./picture/image3.png)

### Multi-Head Attention
上面所定义的一组Q,K,V得到的融合向量只能从一个“视角”去捕捉信息，我们可以定义多组Q,K,V，让每个头关注不同的特则会给你、关系或位置，从而获得更丰富的信息表达。计算 Q,K,V 的过程还是一样，只不过线性变换的矩阵从一组($W^Q,W^K,W^V$)变成了多组$(W_0^Q,W_0^K,W_0^V),(W_1^Q,W_1^Q,W_1^Q)$,..如下图所示

![](./picture/image4.png)

对于输入矩阵 X，每一组 Q、K 和 V 都可以得到一个输出矩阵Z。如下图所示

![](./picture/image5.png)

### Src_input Paddding Mask
对于encodee的源语句输入时，其中每个mini-batch是由多个不等长的句子组成的，我们需要按照这个mini-batch中最大的句长对剩余的句子进行补齐，一般用0进行填充，这个过程叫做 padding但这时在进行 softmax 就会产生问题。回顾softmax函数$\sigma(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$,$e^0$是1，是有值的，这样的话 softmax 中被 padding 的部分就参与了运算，相当于让无效的部分参与了运算，这可能会产生很大的隐患。因此需要做一个 mask 操作，让这些无效的区域不参与运算，一般是给无效区域加一个很大的负数偏置，即
$$
Z_{\text{illegal}} = Z_{\text{illegal}} + \text{bias}_{\text{illegal}}
$$
$$
\text{bias}_{\text{illegal}} \rightarrow -\infty
$$

In [6]:
import math
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout=0.1):
        super().__init__()

        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)

    def attention(self, q, k, v, d_k, mask=None, dropout=None):
        #(166*8*9*64) * (166*8*64*9) --> (166*8*9*9)  
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)

        # 此时mask的维度为(166*1*8)
        # 掩盖那些为了补全长度而增加的单元，使其通过Softmax计算后为0
        if mask is not None:
            mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)
        scores = F.softmax(scores, dim=-1)
        if dropout is not None:
            scores = dropout(scores)
        
        output = torch.matmul(scores, v)
        return output
    
    def forward(self, q, k, v, mask=None):
        bs = q.size(0)

        # 为了通过一个线性层实现多个头并行计算，将输入维度减小，即
        # 输入向量的维度为(166*9*512)，经过线性层的维度仍为(166*9*512)，
        # 接着改变特征维度为(166*9*8*64) (batch_size * max_length * heads * d_k)
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)

        # 矩阵转置
        k = k.transpose(1, 2)
        q = q.transpose(1, 2)
        v = v.transpose(1, 2)
        
        #multi_output的维度为（166*8*9*64）
        multi_output = self.attention(q, k, v, self.d_k, mask, dropout=self.dropout)
        
        # 连接多个头并输入最后的线性层 （166*9*512）
        concat = multi_output.transpose(1, 2).contiguous().view(bs, -1, self.d_model)

        output = self.out(concat)

        return output

    

In [7]:
#可重复的编码器层
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        
        self.attn = MultiHeadAttention(heads, d_model, dropout=dropout)


    def forward(self, x, mask):
        x = self.attn(x, x, x, mask)
        return x

In [8]:
class Encoder(nn.Module):
    def __init__(self, d_model, heads, N, vocab_size, max_length, dropout):
        super().__init__()
        self.d_model = d_model
        self.heads = heads
        self.N = N
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.dropout = dropout
        self.embedding = Embedding(vocab_size, d_model)
        self.pos_encoding = positionalEncoding(d_model, max_length)
        self.layers = get_clones(EncoderLayer(d_model, heads, dropout), N)
        #self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embedding(src)
        x = self.pos_encoding(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)  
        return x

In [9]:
##测试代码
def create_mask(src, src_pad):
    src_mask = (src != src_pad).unsqueeze(-2)
    return src_mask

encoder = Encoder(d_model, heads, N, src_vocab, max_strlen, dropout)
for i, batch in enumerate(train_iter):
    src = batch.src.transpose(0, 1)
    src_mask = create_mask(src, src_pad)
    pred = encoder(src, src_mask)
    if(i == 1):
        break
pred.shape


torch.Size([166, 5, 512])